In [4]:
import csv
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np

# Read data from an Excel file
data = pd.read_excel('awards_until2019_imdbid.xlsx')
movies = data['imdbid'].tolist()  # Get a list of movie IDs
base_url = 'https://www.imdb.com'  # Base URL for IMDb website
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}  # User-Agent header to be used in requests
reviews_data = []  # List to store the scraped reviews

# Function to extract reviews from a given URL
def extract_reviews(url, movie_id):
    try:
        try:
            movie_name = get_movie_name(movie_id)
        except:
            movie_name = 'Nan'  # Set movie_name to 'Nan' if it fails to retrieve the movie name
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        base = f"{base_url}/title/{movie_id}/reviews"
        review_divs = soup.find_all('div', {'class': 'lister-item-content'})

        reviews = []
        for review_div in review_divs:
            try:
                # Extract relevant information from each review_div
                review_name = review_div.find('a', {'class': 'title'}).text.strip()
                review_content = review_div.find('div', {'class': 'text'}).text.strip()
                reviewer_name = review_div.find('span', {'class': 'display-name-link'}).text.strip()
                reviewer_link = base_url + review_div.find('span', {'class': 'display-name-link'}).a['href']
                review_link = base_url + review_div.find('a', {'class': 'title'})['href']
                review_date = review_div.find('span', {'class': 'review-date'}).text.strip()
                review_rating = review_div.find('span', {'class': 'rating-other-user-rating'}).find('span').text.strip()

                # Store the extracted information in a dictionary and append to the reviews list
                reviews.append({
                    'movie_id': movie_id,
                    'movie_name': movie_name,
                    'title': review_name,
                    'link': review_link,
                    'date': review_date,
                    'rating': review_rating,
                    'reviewer_name': reviewer_name,
                    'reviewer_link': reviewer_link,
                    'review_content': review_content
                })
            except AttributeError:
                continue
    except:
        # If any exception occurs during scraping, recursively call the function again
        extract_reviews(url, movie_id)

    # Check if there are additional pages and scrape them
    load_more_div = soup.find('div', {'class': 'load-more-data'})
    if load_more_div is not None and 'data-key' in load_more_div.attrs:
        load_more_link = load_more_div['data-key']
        new_url = f"{base}/_ajax?paginationKey={load_more_link}"
        reviews.extend(extract_reviews(new_url, movie_id))

    return reviews

# Function to get the movie name
def get_movie_name(movie_id):
    response = requests.get(f"{base_url}/title/{movie_id}/", headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup.find('section', {'class': 'ipc-page-section'}).h1.text

pbar = tqdm(total=len(movies), desc='Scraping Reviews')
#existing_ids=pd.read_csv('reviews_data.csv')['movie_id'].unique()
#existing_ids2=pd.read_csv('reviews_data2.csv')['movie_id'].unique()
#existing_ids3=pd.read_csv('reviews_data3.csv')['movie_id'].unique()
#existing_ids4=pd.read_csv('reviews_data4.csv')['movie_id'].unique()
#existing=np.concatenate((existing_ids,existing_ids2,existing_ids3,existing_ids4))


for movie_id in movies :
    #if movie_id not in existing :
        url = f"{base_url}/title/{movie_id}/reviews"
        movie_reviews = extract_reviews(url, movie_id)
        print('For film with ID', movie_id, len(movie_reviews), 'reviews were scraped')
        pbar.update(1)
        reviews_data.extend(movie_reviews)
        # Convert the list of dictionaries into a DataFrame
        df = pd.DataFrame(reviews_data)
        # Convert the 'date' column to datetime
        try :
            df['date'] = pd.to_datetime(df['date'])
        except :
            pass
        # Save the DataFrame to a CSV file
        df.to_csv('reviews_data*.csv', index=False)
        


Scraping Reviews:   0%|          | 1/667 [00:05<1:01:39,  5.56s/it]

For film with ID tt0113254 0 reviews were scraped


Scraping Reviews:   0%|          | 2/667 [00:08<41:45,  3.77s/it]  

For film with ID tt0240885 0 reviews were scraped


Scraping Reviews:   0%|          | 3/667 [01:08<5:29:49, 29.80s/it]

...

For film with ID tt6412452 782 reviews were scraped


Scraping Reviews:  57%|█████▋    | 383/667 [4:40:38<6:10:59, 78.38s/it] 

For film with ID tt7125860 311 reviews were scraped


Scraping Reviews:  58%|█████▊    | 384/667 [4:41:14<5:10:41, 65.87s/it]

For film with ID tt6053438 393 reviews were scraped


In [5]:
df

,movie_id,movie_name,title,link,date,rating,reviewer_name,reviewer_link,review_content
0,tt0387131,The Constant Gardener,"3-in-1 Film: Romance, Thriller, and Social Rea...",https://www.imdb.com/review/rw1163018/?ref_=tt...,2005-09-01,8,lavatch,https://www.imdb.com/user/ur2767381/?ref_=tt_urv,"""The Constant Gardener"" seeks to juggle three ..."
1,tt0387131,The Constant Gardener,Thought Provoking and Beautifully Stirring Pol...,https://www.imdb.com/review/rw1166746/?ref_=tt...,2005-09-06,10,WriterDave,https://www.imdb.com/user/ur1069062/?ref_=tt_urv,Ralph Fiennes stars as a British diplomat whos...
2,tt0387131,The Constant Gardener,CONSTANCY Inevitably Unearths the Truth!,https://www.imdb.com/review/rw3517969/?ref_=tt...,2016-08-02,9,KissEnglishPasto,https://www.imdb.com/user/ur47239035/?ref_=tt_urv,"From PASTO, Colombia-Via: L. A. CA; CALI, COLO..."
3,tt0387131,The Constant Gardener,great,https://www.imdb.com/review/rw3560669/?ref_=tt...,2016-10-12,8,SnoopyStyle,https://www.imdb.com/user/ur2898520/?ref_=tt_urv,Justin Quayle (Ralph Fiennes) is a mild-manner...
4,tt0387131,The Constant Gardener,Great story.... and sad truths about Big Pharma,https://www.imdb.com/review/rw7652435/?ref_=tt...,2021-12-17,8,tmendonca-50956,https://www.imdb.com/user/ur98824700/?ref_=tt_urv,"""Put $50,000 in the right hands and you can te..."
...,...,...,...,...,...,...,...,...,...
190404,tt6053438,Sur le chemin de la rédemption,A terrible movie,https://www.imdb.com/review/rw4304637/?ref_=tt...,2018-08-26,1,maxwell-58088,https://www.imdb.com/user/ur91662753/?ref_=tt_urv,I thought this would be a good movie based on ...
190405,tt6053438,Sur le chemin de la rédemption,First Reformed,https://www.imdb.com/review/rw6491180/?ref_=tt...,2021-01-17,10,mrls-05315,https://www.imdb.com/user/ur61987197/?ref_=tt_urv,"I couldn't help comparing this movie to ""Winte..."
190406,tt6053438,Sur le chemin de la rédemption,Less Taxi Driver and more The Believer with be...,https://www.imdb.com/review/rw8629961/?ref_=tt...,2022-10-23,8,trey-89891,https://www.imdb.com/user/ur63444148/?ref_=tt_urv,Perfect performances and truly engaging direct...
190407,tt6053438,Sur le chemin de la rédemption,Just terrible,https://www.imdb.com/review/rw4512751/?ref_=tt...,2018-12-15,2,lachlankennedy,https://www.imdb.com/user/ur96402918/?ref_=tt_urv,I'll give the movie points for the tension cre...
